# Main Script HiFiGAN

### Imports

In [1]:
try: 
    import librosa
except:
    !pip install librosa
try: 
    import optuna, plotly
except:
    !pip install optuna
    !pip install plotly
try:
    from audio_diffusion_pytorch import DiffusionVocoder, UNetV0, LTPlugin, VDiffusion, VSampler
except:
    !pip install audio-diffusion-pytorch
#Set Dir 
import sys, os
sys.path.append(os.path.abspath('..'))

# Torch
import torch
from torch import nn, Tensor
from torch.utils.data import DataLoader, Subset
from torch.nn.utils import weight_norm 
from torch.nn import utils
import torch.optim as optim
import torch.nn.functional as F
import optuna, plotly


# Utils
import numpy as np
from numpy import ndarray
import logging, librosa, itertools, tensorboard
from typing import Sequence, Optional, Callable
from audio_diffusion_pytorch import DiffusionVocoder, UNetV0, LTPlugin, VDiffusion, VSampler


# Base Scripts
from Libraries.Utils import *
from MainScripts.Conf import conf

### Config

### General

In [2]:
remote_kernel: bool = True #Set to true if using a remote Kernel changes the file structure
model_name: str = "Diffusion_Vocoder_v1"
training_label_name: str = "training_full_wave"
full_model_path: str = path_to_remote_path("{}/{}".format(conf["paths"].model_path, model_name + ".pth"), remote_kernel)

Logging

In [3]:
logging_level: int = LIGHT_DEBUG
logging.basicConfig(level=logging_level, format='%(asctime)s - %(levelname)s - %(message)s')
logger: logging.Logger = logging.getLogger(__name__)

Training Params

In [4]:
device: str = "cuda" if torch.cuda.is_available() else "cpu"
n_training_samples: int = 2496 // 2
n_val_samples: int = 200
batch_size: int = 8
tensor_wave_dim: list = [batch_size, 1, 2**17] #B, C, H = Batch, channels, Time domain
tensor_mel_dim: list = [batch_size, 96, 512]
learning_rate: float = 2e-4
betas = [0.8, 0.99]
epochs: int = 300
restart_training: bool = True
checkpoint_freq: int = 5
num_workers: int = 4


### Data Loading

In [5]:
audio_data: ndarray = load_training_data(path_to_remote_path("{}/{}".format(conf["paths"].data_path, training_label_name + ".npy"), remote_kernel))

np.random.seed(50)
indicies: ndarray = np.arange(audio_data.shape[0])
np.random.shuffle(indicies)
audio_data = audio_data[indicies]

data_loader = create_dataloader(Audio_Data(audio_data[:n_training_samples]), batch_size, num_workers)
validation_dataloader = create_dataloader(Audio_Data(audio_data[:-n_val_samples]), batch_size, num_workers)

2025-06-23 17:03:40,510 - LIGHT_DEBUG - Ndarray loaded from Data/training_full_wave.npy of shape: (6867, 131072)


### Original Impl

In [6]:
vocoder = DiffusionVocoder(
    mel_n_fft=1024,
    mel_channels=96,
    mel_sample_rate=32000,
    mel_normalize_log=True, 
    net_t=UNetV0,
    channels=[128, 128, 256, 512],
    factors=[2, 2, 4, 4],
    items=[4, 4, 6, 6],
    diffusion_t=VDiffusion,
    sampler_t=VSampler,
).to(device)

Model Loading

In [7]:
optimizer = optim.AdamW(vocoder.parameters(), lr=learning_rate, betas=betas)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.98)
start_epoch: int = 0
if os.path.exists(full_model_path):
    model = torch.load(full_model_path, map_location=device)
    vocoder.load_state_dict(model["model"])
    if not restart_training:
        optimizer.load_state_dict(model["optim"])
        scheduler.load_state_dict(model["scheduler"])
        start_epoch = model.get("epoch", 0)
    logger.info(f"Model {model_name} loaded with {count_parameters(vocoder)} Parameters")
else: 
    logger.info(f"Model {model_name} created with {count_parameters(vocoder)} Parameters")

2025-06-23 17:03:47,759 - INFO - Model Diffusion_Vocoder_v1 created with ~51.49M Parameters


Initiate Trainer

In [8]:
trainer = Trainer(vocoder, optimizer, scheduler, device, n_dims=1)

### Training

In [ ]:
train_loss, val_loss = trainer.train(train_dataset=data_loader,
            val_dataset=validation_dataloader,
            n_epochs=100,
            full_model_path=full_model_path,
            checkpoint_freq=checkpoint_freq,
            patience=-1,
            gradient_clip_norm=None,
            gradient_clip_val=None,
            sample_freq=None,
            )
scatter_plot(train_loss)
scatter_plot(val_loss)

2025-06-23 17:03:47,952 - INFO - Training started on cuda


2025-06-23 17:06:32,000 - LIGHT_DEBUG - Batch 156/156 Loss: 0.100 Min/Max params: -2.313, 2.126


2025-06-23 17:11:42,971 - INFO - Epoch 001: Avg. Loss: 2.16561e-01 Avg. val. Loss: 1.38616e-01 Remaining Time: 13h 03min 46s LR: 1.96000e-04 


2025-06-23 17:14:27,000 - LIGHT_DEBUG - Batch 156/156 Loss: 0.082 Min/Max params: -2.313, 2.125


2025-06-23 17:19:37,617 - INFO - Epoch 002: Avg. Loss: 1.26975e-01 Avg. val. Loss: 1.23887e-01 Remaining Time: 12h 55min 33s LR: 1.92080e-04 


...

2025-06-23 19:28:52,000 - LIGHT_DEBUG - Batch 156/156 Loss: 0.104 Min/Max params: -2.338, 2.145


2025-06-23 19:34:02,466 - INFO - Epoch 019: Avg. Loss: 1.24813e-01 Avg. val. Loss: 1.25734e-01 Remaining Time: 10h 40min 22s LR: 1.36247e-04 


2025-06-23 19:36:46,000 - LIGHT_DEBUG - Batch 156/156 Loss: 0.081 Min/Max params: -2.339, 2.145


2025-06-23 19:41:56,910 - INFO - Epoch 020: Avg. Loss: 1.40714e-01 Avg. val. Loss: 1.27977e-01 Remaining Time: 10h 32min 28s LR: 1.33522e-04 
2025-06-23 19:41:57,548 - LIGHT_DEBUG - Models/Diffusion_Vocoder_v1_epoch_015.pth deleted
2025-06-23 19:41:57,549 - LIGHT_DEBUG - Checkpoint saved model to Models/Diffusion_Vocoder_v1_epoch_020.pth


2025-06-23 19:44:19,000 - LIGHT_DEBUG - Batch 135/156 Loss: 0.136 Min/Max params: -2.338, 2.145